In [ ]:
# Перевод данных из датасета в массивы

import pandas as pd
import numpy as np

h, w = 48, 48

dataset_dir = 'datasets/fer2013.csv'
dataset_plus_dir = 'datasets/fer2013new.csv'
dataset = pd.read_csv(dataset_dir)
dataset_plus = pd.read_csv(dataset_plus_dir)

drop_indices = dataset_plus.index[
#       (dataset_plus['neutral'] < 10) &
      (dataset_plus['happiness'] < 10)
    & (dataset_plus['surprise'] < 9)
    & (dataset_plus['sadness'] < 7)
    & (dataset_plus['anger'] < 7)
#     & (dataset_plus['disgust'] < 10)
#     & (dataset_plus['fear'] < 10)
    | (dataset_plus['contempt'] > 1)
    | (dataset_plus['unknown'] > 1)
    | (dataset_plus['NF'] > 0)]
dataset = dataset.drop(drop_indices, axis=0)
drop_indices = dataset.index[(dataset['emotion'] == 1) | (dataset['emotion'] == 2) | (dataset['emotion'] == 6)]
dataset = dataset.drop(drop_indices, axis=0)
drop_indices = np.random.choice(dataset.index[dataset['emotion'] == 3], 3800, replace=False)
dataset = dataset.drop(drop_indices, axis=0)

# num_train_samples = len(dataset[dataset['Usage'] == 'Training'])
# num_val_samples = len(dataset[dataset['Usage'] == 'PublicTest'])
# num_test_samples = len(dataset[dataset['Usage'] == 'PrivateTest'])

num_train_samples = len(dataset) - 2000
num_val_samples = 1000
num_test_samples = 1000

train_images = np.zeros((num_train_samples, h, w))
validation_images = np.zeros((num_val_samples, h, w))
test_images = np.zeros((num_test_samples, h, w))

train_labels = np.zeros(num_train_samples)
validation_labels = np.zeros(num_val_samples)
test_labels = np.zeros(num_test_samples)

# i, j, k = 0, 0, 0
# for index, row in dataset.iterrows():
    
#     if row['Usage'] == 'Training':
#         train_images[i] = np.fromstring(row['pixels'], dtype=np.uint8, sep=' ').reshape(h, w)
#         train_labels[i] = row['emotion']
#         i += 1

#     elif row['Usage'] == 'PublicTest':
#         validation_images[j] = np.fromstring(row['pixels'], dtype=np.uint8, sep=' ').reshape(h, w)
#         validation_labels[j] = row['emotion']
#         j += 1

#     elif row['Usage'] == 'PrivateTest':
#         test_images[k] = np.fromstring(row['pixels'], dtype=np.uint8, sep=' ').reshape(h, w)
#         test_labels[k] = row['emotion']
#         k += 1

i, j, k, f = 0, 0, 0, 0
for index, row in dataset.iterrows():
    
    if (i < 3000) | (i >= 5000):
        train_images[j] = np.fromstring(row['pixels'], dtype=np.uint8, sep=' ').reshape(h, w)
        train_labels[j] = row['emotion']
        j += 1

    elif (i >= 3000) & (i < 4000):
        validation_images[k] = np.fromstring(row['pixels'], dtype=np.uint8, sep=' ').reshape(h, w)
        validation_labels[k] = row['emotion']
        k += 1

    elif (i >= 4000) & (i < 5000):
        test_images[f] = np.fromstring(row['pixels'], dtype=np.uint8, sep=' ').reshape(h, w)
        test_labels[f] = row['emotion']
        f += 1
        
    i += 1

In [ ]:
# Подготовка данных к передаче в нейросеть

from keras.utils import to_categorical

train_images = np.kron(train_images, np.ones((2, 2)))
validation_images = np.kron(validation_images, np.ones((2, 2)))
test_images = np.kron(test_images, np.ones((2, 2)))

train_images = np.stack((train_images,) * 3, 3)
validation_images = np.stack((validation_images,) * 3, 3)
test_images = np.stack((test_images,) * 3, 3)

train_images = train_images.astype('float32') / 255
validation_images = validation_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

i = 0
for label in train_labels:
    if label > 2:
        train_labels[i] -= 2
    i += 1

i = 0
for label in validation_labels:
    if label > 2:
        validation_labels[i] -= 2
    i += 1

i = 0
for label in test_labels:
    if label > 2:
        test_labels[i] -= 2
    i += 1

train_labels = to_categorical(train_labels)
validation_labels = to_categorical(validation_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Создание генераторов

from keras.preprocessing.image import ImageDataGenerator

batch_size = 64

train_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()

train_gen = train_gen.flow(
    train_images,
    train_labels,
    batch_size = batch_size)

validation_gen = validation_gen.flow(
    validation_images,
    validation_labels,
    batch_size = batch_size)

test_gen = test_gen.flow(
    test_images,
    test_labels,
    batch_size = batch_size)

In [ ]:
# Замороженная VGG16

from keras import models
from keras import layers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.applications import VGG16
import math

number_of_epochs = 30

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in vgg16.layers:
    layer.trainable = False

model = models.Sequential()
model.add(vgg16)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_vgg16_fr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_vgg16_fr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Размороженная VGG16

number_of_epochs = 70

set_trainable = False
for layer in vgg16.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_vgg16_unfr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_vgg16_unfr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Замороженная VGG19

from keras import models
from keras import layers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.applications import VGG19
import math

number_of_epochs = 30

vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
for layer in vgg19.layers:
    layer.trainable = False

model = models.Sequential()
model.add(vgg19)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_vgg19_fr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_vgg19_fr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Размороженная VGG19

number_of_epochs = 70

set_trainable = False
for layer in vgg19.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_vgg19_unfr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_vgg19_unfr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Замороженная ResNet50

from keras import models
from keras import layers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.applications import ResNet50
import math

number_of_epochs = 30

resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
resnet50.trainable = False

model = models.Sequential()
model.add(resnet50)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_resnet50_fr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_resnet50_fr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Размороженная ResNet50

number_of_epochs = 70

resnet50.trainable = True
set_trainable = False
for layer in resnet50.layers:
    if layer.name == 'res3a_branch2a':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_resnet50_unfr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_resnet50_unfr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Замороженная InceptionV3

from keras import models
from keras import layers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.applications import InceptionV3
import math

number_of_epochs = 30

inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
inceptionv3.trainable = False

model = models.Sequential()
model.add(inceptionv3)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=2e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_inceptionv3_fr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_inceptionv3_fr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Размороженная InceptionV3

number_of_epochs = 70

inceptionv3.trainable = True
# set_trainable = False
# for layer in inceptionv3.layers:
#     if layer.name == 'mixed6':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_inceptionv3_unfr.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model = load_model('ferCNN_inceptionv3_unfr.h5')

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Выводит графики точности и потерь

%matplotlib inline
import matplotlib.pyplot as plt

train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig('acc.png', bbox_inches='tight')
plt.savefig('acc.pdf', bbox_inches='tight')
plt.figure()

plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig('loss.png', bbox_inches='tight')
plt.savefig('loss.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Выводит сглаженные графики точности и потерь

%matplotlib inline
import matplotlib.pyplot as plt

def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

plt.plot(epochs, smooth_curve(train_acc), 'bo', label='Smoothed training acc')
plt.plot(epochs, smooth_curve(val_acc), 'r', label='Smoothed validation acc')
plt.title('Smoothed training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.savefig('smoothed_acc.png', bbox_inches='tight')
plt.savefig('smoothed_acc.pdf', bbox_inches='tight')
plt.figure()

plt.plot(epochs, smooth_curve(train_loss), 'bo', label='Smoothed training loss')
plt.plot(epochs, smooth_curve(val_loss), 'r', label='Smoothed validation loss')
plt.title('Smoothed training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig('smoothed_loss.png', bbox_inches='tight')
plt.savefig('smoothed_loss.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Тестирование нейросети

from keras.models import load_model
import math

model = load_model('ferCNN_vgg16_fr.h5')
test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))

print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Готовое приложение emotion_recognizer_v1.1.py

# %%writefile emotion_recognizer.py
from PyQt5 import QtCore, QtGui, QtWidgets
import ntpath
import os

class Ui_MainWindow(object):
    model_path, image_path = "", ""

    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(600, 500)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.gridLayout = QtWidgets.QGridLayout(self.centralwidget)
        self.gridLayout.setObjectName("gridLayout")
        self.selectedImageLbl = QtWidgets.QLabel(self.centralwidget)
        self.selectedImageLbl.setMinimumSize(QtCore.QSize(150, 150))
        self.selectedImageLbl.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.selectedImageLbl.setText("")
        self.selectedImageLbl.setObjectName("selectedImageLbl")
        self.gridLayout.addWidget(self.selectedImageLbl, 0, 0, 1, 4)
        self.modelTextLbl = QtWidgets.QLabel(self.centralwidget)
        self.modelTextLbl.setMaximumSize(QtCore.QSize(16777215, 30))
        font = QtGui.QFont()
        font.setPointSize(11)
        self.modelTextLbl.setFont(font)
        self.modelTextLbl.setAlignment(QtCore.Qt.AlignCenter)
        self.modelTextLbl.setObjectName("modelTextLbl")
        self.gridLayout.addWidget(self.modelTextLbl, 2, 0, 1, 1)
        self.emotionTextLbl = QtWidgets.QLabel(self.centralwidget)
        self.emotionTextLbl.setMaximumSize(QtCore.QSize(16777215, 30))
        font = QtGui.QFont()
        font.setPointSize(11)
        self.emotionTextLbl.setFont(font)
        self.emotionTextLbl.setAlignment(QtCore.Qt.AlignCenter)
        self.emotionTextLbl.setObjectName("emotionTextLbl")
        self.gridLayout.addWidget(self.emotionTextLbl, 1, 0, 1, 1)
        self.modelNameLbl = QtWidgets.QLabel(self.centralwidget)
        self.modelNameLbl.setMaximumSize(QtCore.QSize(16777215, 30))
        font = QtGui.QFont()
        font.setPointSize(11)
        self.modelNameLbl.setFont(font)
        self.modelNameLbl.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.modelNameLbl.setText("")
        self.modelNameLbl.setObjectName("modelNameLbl")
        self.gridLayout.addWidget(self.modelNameLbl, 2, 1, 1, 3)
        self.emotionNameLbl = QtWidgets.QLabel(self.centralwidget)
        self.emotionNameLbl.setMaximumSize(QtCore.QSize(16777215, 30))
        font = QtGui.QFont()
        font.setPointSize(11)
        self.emotionNameLbl.setFont(font)
        self.emotionNameLbl.setFrameShape(QtWidgets.QFrame.StyledPanel)
        self.emotionNameLbl.setText("")
        self.emotionNameLbl.setObjectName("emotionNameLbl")
        self.gridLayout.addWidget(self.emotionNameLbl, 1, 1, 1, 3)
        self.selectImageBtn = QtWidgets.QPushButton(self.centralwidget)
        self.selectImageBtn.setMinimumSize(QtCore.QSize(0, 35))
        font = QtGui.QFont()
        font.setPointSize(11)
        self.selectImageBtn.setFont(font)
        self.selectImageBtn.setObjectName("selectImageBtn")
        self.gridLayout.addWidget(self.selectImageBtn, 3, 0, 1, 2)
        self.recognizeEmotionBtn = QtWidgets.QPushButton(self.centralwidget)
        self.recognizeEmotionBtn.setMinimumSize(QtCore.QSize(0, 35))
        font = QtGui.QFont()
        font.setPointSize(11)
        self.recognizeEmotionBtn.setFont(font)
        self.recognizeEmotionBtn.setObjectName("recognizeEmotionBtn")
        self.gridLayout.addWidget(self.recognizeEmotionBtn, 3, 2, 1, 2)
        MainWindow.setCentralWidget(self.centralwidget)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        self.selectImageBtn.clicked.connect(self.setImage)
        self.recognizeEmotionBtn.clicked.connect(self.recognizeEmotion)

        for file in os.listdir(sys.path[0]):
            file_path = os.path.join(sys.path[0], file)
            if os.path.isfile(file_path) & (file.endswith('.h5')):
                self.model_path = file_path
                self.modelNameLbl.setText(file)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "Распознаватель эмоций"))
        self.modelTextLbl.setText(_translate("MainWindow", "Модель:"))
        self.emotionTextLbl.setText(_translate("MainWindow", "Эмоция:"))
        self.selectImageBtn.setText(_translate("MainWindow", "Выбрать изображение"))
        self.recognizeEmotionBtn.setText(_translate("MainWindow", "Распознать эмоцию"))

    def setImage(self):
        imagePath, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select Image", "", "Image Files (*.png *.jpg *.jpeg *.bmp)")
        if imagePath:
            self.image_path = imagePath
            pixmap = QtGui.QPixmap(imagePath)
            pixmap = pixmap.scaled(self.selectedImageLbl.width(), self.selectedImageLbl.height(), QtCore.Qt.KeepAspectRatio)
            self.selectedImageLbl.setPixmap(pixmap)
            self.selectedImageLbl.setAlignment(QtCore.Qt.AlignCenter)

    def recognizeEmotion(self):
        if self.model_path and self.image_path:
            from keras.preprocessing import image
            from keras.models import load_model
            import numpy as np
            import datetime
            emotions = ('злость', 'счастье', 'грусть', 'удивление')

            img = image.load_img(self.image_path, target_size = (96, 96))
            image_array = image.img_to_array(img)
            image_array = np.expand_dims(image_array, axis = 0) / 255

            model = load_model(self.model_path)
            predictions = model.predict(image_array)
            max_emotion_percent = '{:.2f}'.format(np.amax(predictions) * 100)
            max_emotion_index = np.argmax(predictions)
            recognized_emotion = emotions[max_emotion_index].capitalize() + ' (' + max_emotion_percent + '%)'
            self.emotionNameLbl.setText(recognized_emotion)

            with open(os.path.join(sys.path[0], "emotions_log.txt"), "a+") as f:
                now = datetime.datetime.now()
                f.write('Модель: ' + ntpath.basename(self.model_path) +
                        ';   Изображение: ' + ntpath.basename(self.image_path) +
                        ';   Эмоция: ' + recognized_emotion +
                        ';   Дата: ' + now.strftime("%Y-%m-%d %H:%M:%S") + '\n')
                f.close()

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    app.exec_()

In [ ]:
# Код основной логики приложения emotion_recognizer.py

# import ntpath
# import os


#     model_path, image_path = "", ""


# #         self.selectModelBtn.clicked.connect(self.setModel)
#         self.selectImageBtn.clicked.connect(self.setImage)
#         self.recognizeEmotionBtn.clicked.connect(self.recognizeEmotion)

#         for file in os.listdir(sys.path[0]):
#             file_path = os.path.join(sys.path[0], file)
#             if os.path.isfile(file_path) & (file.endswith('.h5')):
#                 self.model_path = file_path
#                 self.modelNameLbl.setText(file)


# #     def setModel(self):
# #         modelPath, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select Model", "", "FER Models (*.h5)")
# #         if modelPath:
# #             self.model_path = modelPath
# #             self.modelNameLbl.setText(ntpath.basename(modelPath))

#     def setImage(self):
#         imagePath, _ = QtWidgets.QFileDialog.getOpenFileName(None, "Select Image", "", "Image Files (*.png *.jpg *.jpeg *.bmp)")
#         if imagePath:
#             self.image_path = imagePath
#             pixmap = QtGui.QPixmap(imagePath)
#             pixmap = pixmap.scaled(self.selectedImageLbl.width(), self.selectedImageLbl.height(), QtCore.Qt.KeepAspectRatio)
#             self.selectedImageLbl.setPixmap(pixmap)
#             self.selectedImageLbl.setAlignment(QtCore.Qt.AlignCenter)

#     def recognizeEmotion(self):
#         if self.model_path and self.image_path:
#             from keras.preprocessing import image
#             from keras.models import load_model
#             import numpy as np
#             import datetime
#             emotions = ('злость', 'счастье', 'грусть', 'удивление')

#             img = image.load_img(self.image_path, target_size = (96, 96))
#             image_array = image.img_to_array(img)
#             image_array = np.expand_dims(image_array, axis = 0) / 255

#             model = load_model(self.model_path)
#             predictions = model.predict(image_array)
#             max_emotion_percent = '{:.2f}'.format(np.amax(predictions) * 100)
#             max_emotion_index = np.argmax(predictions)
#             recognized_emotion = emotions[max_emotion_index].capitalize() + ' (' + max_emotion_percent + '%)'
#             self.emotionNameLbl.setText(recognized_emotion)

#             with open(os.path.join(sys.path[0], "emotions_log.txt"), "a+") as f:
#                 now = datetime.datetime.now()
#                 f.write('Модель: ' + ntpath.basename(self.model_path) +
#                         ';   Изображение: ' + ntpath.basename(self.image_path) +
#                         ';   Эмоция: ' + recognized_emotion +
#                         ';   Дата: ' + now.strftime("%Y-%m-%d %H:%M:%S") + '\n')
#                 f.close()


#     app.exec_()

In [ ]:
# Полезный код

# %load emotion_recognizer.py
# %%writefile emotion_recognizer.py
# pyuic5 -x C:\Users\TheDeGe\Desktop\emotion_recognizer.ui -o C:\Users\TheDeGe\Desktop\emotion_recognizer.py

In [ ]:
# Обычная CNN

from keras import models
from keras import layers
from keras import optimizers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import math

number_of_epochs = 100

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

checkpoint = ModelCheckpoint('ferCNN_simple.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

# history = model.fit(
#     train_images,
#     train_labels,
#     epochs=number_of_epochs,
#     batch_size=batch_size,
#     validation_data=(validation_images, validation_labels))

history = model.fit_generator(
    train_gen,
    steps_per_epoch=math.ceil(num_train_samples / batch_size),
    epochs=number_of_epochs,
    validation_data=validation_gen,
    validation_steps=math.ceil(num_val_samples / batch_size),
    callbacks=[checkpoint])

# model.save('ferCNN_simple.h5')

# val_loss, val_acc = model.evaluate(validation_images, validation_labels)
# print('val_acc:', val_acc)
# print('val_loss:', val_loss)

test_loss, test_acc = model.evaluate_generator(test_gen, steps=math.ceil(num_test_samples / batch_size))
print('test_acc:', test_acc)
print('test_loss:', test_loss)

In [ ]:
# Расшифровка меток эмоций: 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
# Отображает информацию о датасете

# import pandas as pd

# dataset_dir = 'datasets/fer2013.csv'
# dataset = pd.read_csv(dataset_dir)

# print(dataset.head())
# print('\n')
# print(dataset["Usage"].value_counts())
# print('\n')
# print(dataset["emotion"].value_counts())

In [ ]:
# Предсказывает эмоцию по изображению и выводит график предсказаний

# %matplotlib inline
# from keras.preprocessing import image
# from keras.models import load_model
# import matplotlib.pyplot as plt
# import numpy as np

# model_path = 'ferCNN_vgg16_fr.h5'
# image_path = '123.jpg'

# def emotion_analysis(predictions):
#     emotions = ('angry', 'happy', 'sad', 'surprise')
#     max_em_index = np.argmax(predictions)
#     y_pos = np.arange(len(emotions))
    
#     plt.bar(y_pos, predictions[0], align='center', alpha=0.5)
#     plt.xticks(y_pos, emotions)
#     plt.ylabel('Percentage')
#     plt.title('Emotion: ' + emotions[max_em_index].capitalize())
# #     plt.savefig('emotion_analysis.png', bbox_inches='tight')
#     plt.show()

# full_image = image.load_img(image_path)
# img = full_image.resize((96, 96))
# image_array = image.img_to_array(img)
# image_array = np.expand_dims(image_array, axis = 0) / 255

# model = load_model(model_path)
# predictions = model.predict(image_array)
# emotion_analysis(predictions)

# plt.imshow(full_image)
# plt.axis('off')
# plt.show()

In [ ]:
# Отображает 4 варианта расширения данных первого изображения из датасета

# %matplotlib inline
# from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image
# import matplotlib.pyplot as plt

# imagegen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

# images_for_imagegen = train_images[0].reshape((1,) + train_images[0].shape)
# i = 0

# for batch in imagegen.flow(images_for_imagegen, batch_size=1):
#     plt.figure(i)
#     imgplot = plt.imshow(image.array_to_img(batch[0]))
#     plt.axis("off")
#     i += 1
#     if i % 4 == 0:
#         break

# plt.show()

In [ ]:
# Отображает изображения из датасета

# %matplotlib inline
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np

# dataset_dir = 'datasets/fer2013.csv'
# dataset_plus_dir = 'datasets/fer2013new.csv'
# dataset = pd.read_csv(dataset_dir)
# dataset_plus = pd.read_csv(dataset_plus_dir)
# emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

# def show_image_and_label(x, y, i):
#     x_reshaped = x.reshape(48,48)
#     plt.imshow(x_reshaped, cmap= "gray", interpolation="nearest")
#     plt.axis("off")
#     plt.show()
#     print(i, emotions[y])

# # drop_indices = dataset_plus.index[
# # #     (dataset_plus['neutral'] < 10) &
# #       (dataset_plus['happiness'] < 10)
# #     & (dataset_plus['surprise'] < 9)
# #     & (dataset_plus['sadness'] < 7)
# #     & (dataset_plus['anger'] < 7)
# # #     & (dataset_plus['disgust'] < 10)
# # #     & (dataset_plus['fear'] < 10)
# #     | (dataset_plus['contempt'] > 1)
# #     | (dataset_plus['unknown'] > 1)
# #     | (dataset_plus['NF'] > 0)]
# # dataset = dataset.drop(drop_indices, axis=0)
# # drop_indices = dataset.index[(dataset['emotion'] == 1) | (dataset['emotion'] == 2) | (dataset['emotion'] == 6)]
# # dataset = dataset.drop(drop_indices, axis=0)
# # drop_indices = np.random.choice(dataset.index[dataset['emotion'] == 3], 3800, replace=False)
# # dataset = dataset.drop(drop_indices, axis=0)

# # number_of_images = 0
# # i = 0
# # for img in dataset["pixels"][:number_of_images]:
# #     val = img.split(" ")
# #     x_pixels = np.array(val, 'float32') / 255
# #     y_emotion = dataset["emotion"][i]
# #     show_image_and_label(x_pixels, y_emotion)
# #     i += 1

# # image_index = 0
# # img = dataset["pixels"][image_index]
# # val = img.split(" ")
# # x_pixels = np.array(val, 'float32') / 255
# # y_emotion = dataset["emotion"][image_index]
# # show_image_and_label(x_pixels, y_emotion)

# i = 0
# for index, row in dataset_plus.iterrows():
#     if row['NF'] == 10:
#         img = dataset["pixels"][index]
#         val = img.split(" ")
#         x_pixels = np.array(val, 'float32') / 255
#         y_emotion = dataset["emotion"][index]
#         show_image_and_label(x_pixels, y_emotion, index)
#         i += 1
# print(i)

In [ ]:
# Переводит данные из датасета в картинки и сохраняет их по трём папкам:
# 'Training', 'PublicTest', 'PrivateTest' внутри четвёртой: 'fer2013_images' рядом c этим файлом

# import os
# import csv
# import numpy as np
# import imageio

# dataset_dir = 'datasets/fer2013.csv'
# h, w = 48, 48
# id = 0

# with open(dataset_dir, 'r') as csvfile:
#     data_reader = csv.reader(csvfile, delimiter =',')
#     headers = next(data_reader)
    
#     for row in data_reader:
#         pixels = row[1].split()
#         usage = row[2]
        
#         image = np.asarray(pixels, dtype= np.uint8).reshape(h, w)
#         stacked_image = np.dstack((image,) * 3)
        
#         image_folder = os.path.join('fer2013_images', usage)
#         if not os.path.exists(image_folder):
#             os.makedirs(image_folder)
            
#         image_file_name = os.path.join(image_folder, str(id) + '.jpg')
#         imageio.imwrite(image_file_name, stacked_image)
        
#         id += 1
#         if id % 1000 == 0:
#             print('Processed {} images'.format(id))
    
#     print("Finished processing {} images".format(id))